### 1. Set up some dependencies

In [ ]:
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git &>/dev/null;
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !git clone https://github.com/edgi-govdata-archiving/EEW-EJScreen.git
    # Geopandas is an open source library for working with geographic data using the
    #   data structures library "pandas" (common in Python for data processing).
    #   (https://geopandas.org/)
    !pip install pygeos &>/dev/null;
    !pip install geopandas  &>/dev/null;
    %mv /content/EEW-EJScreen/state-fips-codes.csv /content
    %mkdir /content/census-shapefiles
    %mv /content/EEW-EJScreen/census-shapefiles/census2010.db /content/census-shapefiles/census2010.db
else:
    print('Not running on CoLab')

### 2. Enter a state and congressional districe

In [ ]:
# Enter the state and CD. Then you can run all.
state = 'KS'
cd = '2'

### 3. Use shapefiles to construct an initial outline map of the district.

In [ ]:
import geopandas
import folium

url = "https://theunitedstates.io/districts/cds/2012/{}-{}/shape.geojson".format( state, str(cd))       
cd_boundary = geopandas.read_file(url)
bounds = cd_boundary.bounds
map=folium.Map(location=((bounds.miny+bounds.maxy)/2., (bounds.minx+bounds.maxx)/2.), zoom_level=4)
# map.fit_bounds(bounds)
w = folium.GeoJson( cd_boundary, name = "Congressional Districts").add_to(map)


### 4. Look up the state's two-digit FIPS code from a local CSV file.

In [ ]:
import pandas as pd
df = pd.read_csv("state-fips-codes.csv")
df = df[df['State_Code'] == state]
state_fips = str(df['FIPS_Code'].iloc[0]).zfill(2)
state_fips


### 5. Use the FIPS_Code for the state to identify the EJ Screen records just for this state.

In [ ]:
from ECHO_modules.get_data import get_echo_data

sql = 'SELECT count(*) FROM "EJSCREEN_2021_USPR"'
df = get_echo_data(sql)

In [ ]:
df

In [ ]:
from ECHO_modules.get_data import get_echo_data

select_columns = '"ID", "P_LDPNT_D2", "P_DSLPM_D2", "P_CANCR_D2", '
select_columns += '"P_RESP_D2", "P_PTRAF_D2", "P_PWDIS_D2", '
select_columns += '"P_PNPL_D2", "P_PRMP_D2", "P_PTSDF_D2", '
select_columns += '"P_OZONE_D2", "P_PM25_D2"'

sql = 'select {} from "EJSCREEN_2021_USPR" where DIV("ID", 10000000000) = {}' 
sql = sql.format(select_columns, state_fips)
ej_state_df = get_echo_data(sql)
# Rename the ID field to match the field in the census data block group.
ej_state_df.rename(columns={'ID':'GEOID'}, inplace=True)
ej_state_df['GEOID'] = ej_state_df['GEOID'].astype(int)
ej_state_df

### 6. Get census shapefiles for all block groups in the state
ej_state_df now contains the EJScreen data for all census block groups in the state.
We want to identify just those in our CD.
For that we need the bounding polygon of the CD (we have that in cd_boundary), and an interior 
point of the census block, for each census block identified by ID in ej_state_df.
The interior point of the census block is (INTPTLAT, INTPTLON) within the census_block_groups table,
in a SQLite database we've created. (See the census-shapefiles.ipynb notebook in the census-shapefiles
directory.)
Note: This will run acceptably fast with Jupyter Notebook on a local computer. SQLite on Google Colab is pretty slow, however. (It took 2 1/2 minutes to load Massachusetts.) Think of ways of making this faster. One way would be to load the content of census2010.db into
the Stony Brook University PostgreSQL database.

In [ ]:
import sqlite3

conn = sqlite3.connect("census-shapefiles/census2010.db")

bg_point_list = []
for index, row in ej_state_df.iterrows():
    # Use row['GEOID'] to look for the block group in the census db.
    sql = 'select GEOID, INTPTLAT, INTPTLON from census_block_groups where GEOID=\'{}\''.format(row['GEOID'])
    c = conn.cursor()
    c.execute(sql)
    block_group = c.fetchone()
    bg_point_list.append(block_group)
conn.close()    

### 7. Identify the EJ Screen records within this CD.

In [ ]:
import geopandas as gpd

bg_points_df = pd.DataFrame(bg_point_list, columns =['GEOID', 'INTPTLAT', 'INTPTLON'])
bg_points_gdf = gpd.GeoDataFrame(bg_points_df, crs='epsg:4269',
                    geometry=gpd.points_from_xy(bg_points_df.INTPTLON, bg_points_df.INTPTLAT))
bg_points_gdf = bg_points_gdf.to_crs(4326)
within_points = gpd.sjoin(bg_points_gdf, cd_boundary, op='within')
within_points

### 8. Plot the census block groups as a sanity check.

In [ ]:
for i in range(0,len(within_points)):
   folium.Marker(
      location=[within_points.iloc[i]['INTPTLAT'], within_points.iloc[i]['INTPTLON']],
      popup=within_points.iloc[i]['GEOID'],
   ).add_to(map)
map

### 9. Filter the EJ Screen records to just those in the congressional district
We have the census block groups in the within_points dataframe, and the EJ screen records for
all census blocks in the state in ej_state_df. Filter ej_state_df where its ID field matches
GEOID in within_points.

In [ ]:
ej_columns = ['GEOID', 'P_LDPNT_D2', 'P_DSLPM_D2', 'P_CANCR_D2', 'P_RESP_D2', 'P_PTRAF_D2', 'P_PWDIS_D2',
     'P_PNPL_D2', 'P_PRMP_D2', 'P_PTSDF_D2', 'P_OZONE_D2', 'P_PM25_D2']

ej_cd_df = within_points.merge(ej_state_df[ej_columns], on='GEOID')
ej_cd_df

### 10. Compute means and standard deviations for each of the 11 EJ Screen metrics
Select only the 11 columns in EJ Screen for national percentiles of the census block group among all groups in the U.S.

In [ ]:
columns = ['P_LDPNT_D2', 'P_DSLPM_D2', 'P_CANCR_D2', 'P_RESP_D2', 'P_PTRAF_D2', 'P_PWDIS_D2', 
           'P_PNPL_D2', 'P_PRMP_D2', 'P_PTSDF_D2', 'P_OZONE_D2', 'P_PM25_D2']
ej_cd_df[columns] = ej_cd_df[columns].apply(pd.to_numeric)
ej_cd_scores = ej_cd_df[columns]
means = pd.DataFrame(ej_cd_scores.mean())
new_columns=['Lead paint', 'Diesel', 'Air toxics cancer', 'Air toxics resp',
             'Traffic', 'Water discharge', 'NPL sites', 'RMP facilities',
             'TSDF facilities', 'Ozone', 'PM2.5']
means = means.set_axis(new_columns, axis=0)

stdev = pd.DataFrame(ej_cd_scores.std())
stdev = stdev.set_axis(new_columns, axis=0)


### 11. Write out the filtered data.

In [ ]:
ej_cd_scores.to_csv('ejscreen-{}{}-percentiles.csv'.format(state, cd))

### 12. Put the scores for the census block groups into 20 5-percent bins. 
Then count how many block groups have gone into each bin.
Assign each bin the value of the mid-point of its range--2.5, 7.5, ... 92.5, 97.5

In [ ]:
df2 = pd.melt(ej_cd_scores)
df2['value2'] = pd.to_numeric(df2['value'], errors='ignore')
df2['bins'] = pd.cut(df2['value2'], bins=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100])

newdf = pd.DataFrame(columns=['metric', 'value', 'count'])
for metric in columns:
    tempdf = df2[df2['variable'] == metric]
    for bin in tempdf['bins'].unique():
        try:
            midpoint = (bin.left + bin.right)/2
            tempdf2 = tempdf[tempdf['bins'] == bin]
            count = 2 * tempdf2['value'].count()
            new_row = {'metric':metric, 'value':midpoint, 'count':float(count)}
            # breakpoint()
            newdf = newdf.append(new_row,ignore_index=True)
        except AttributeError:
            continue
newdf       
    

### 13. Plot the bins of census block group EJ Screen scores
Plot the count of census blocks in the 20 5-percent bins by assigning a dot whose size
is proportional to the count. Also plot a line showing one standard deviation of the
data around the mean. 

This standard deviation is a measure of the inequality within the district--if
all census block groups were equal the standard deviation would be zero; if there are 
many very high (i.e. bad) EJ scores and also many other low EJ scores, the standard
deviation will be large.

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = [8, 5]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

means.plot(kind='bar', alpha=0., yerr = stdev)
# means_plus_stdev.plot(kind='bar', alpha=0.3)
plt.scatter(x=newdf['metric'], y=newdf['value'], sizes=newdf['count'],
           color='Green')

plt.title("EJ Screen Index Percentiles for {}, District {}".format(state,cd))
plt.xlabel("EJ Screen Categories")
plt.ylabel("Census Blocks in National Percentiles\n and Standard Deviation")
ax = plt.gca()
ax.set_xticklabels(labels=new_columns,rotation=60)
# plt.ylim(0,100)
# plt.axhline(y = 50, color = 'r', linestyle = '-')
filename = "graph-{}-{}.png".format(state, cd)
plt.savefig(filename, bbox_inches="tight")

### Below here doesn't currently work.
To do: 
    Compute these std dev figures for all CDs, into ej_screen_cd_stdev_rankings.csv. That is produced by the all_regions.ipynb notebook.
    These are just ideas for techniques.
    

In [ ]:
xdf = pd.read_csv('ej_screen_cd_stdev_rankings.csv')
ydf = xdf[['Lead_Paint.Pct','Diesel.Pct','Air_toxics_cancer.Pct','Air_toxics_resp.Pct',
                   'Traffic.Pct','Water_discharge.Pct','NPL_sites.Pct','RMP_facilities.Pct',
                  'TSDF_facilities.Pct','Ozone.Pct','PM2.5.Pct','State','CD']]
zdf = ydf.loc[ydf['State'] == state]
rank_df = zdf.loc[zdf['CD'] == int(cd)]

In [ ]:
import numpy as np

grades = ['A','B','C','D','F']

columns = list(rank_df)
for i in columns:
    nm = rank_df[i].name
    if nm != 'State' and nm != 'CD':
        s = rank_df[i]
        value = s[s.index[0]]
        grade = 'None'
        if np.issubdtype(rank_df[i].dtype, np.number):
            value = int(value/0.2)
            grade = grades[value]
        print('{} - Grade for "fairness" in the district is {} ({} percentile nationally)'.format(
            nm, grade, int(s[s.index[0]]*100)))


In [ ]:
print("{: >30}".format("Hansen"))